# Формирование датасета для обучения НС
Подготовка данных для обучения. На этом этапе выполняется формирование набора обрезанных изображений с детекциями людей.

In [1]:
import sys
from pathlib import Path

import tqdm

sys.path.append('/app/src')
import glob
import os
import random
import shutil
from pathlib import Path

import cv2
import yaml
from sklearn.model_selection import train_test_split

from markup_convertors import yolo2bb
from transforms import crop

In [2]:
# Обрезка

DATASET_VERSION = "v8"

people_img_path = "/app/data/interim_v3/images/train/"
labels_img_path = "/app/data/interim_v3/labels/train/"

people_save_path = f"/app/data/processed/{DATASET_VERSION}/images_{DATASET_VERSION}/"
labels_save_path = f"/app/data/processed/{DATASET_VERSION}/labels_{DATASET_VERSION}/"

people_back_save_path = f"/app/data/processed/{DATASET_VERSION}/images_back_{DATASET_VERSION}/"
labels_back_save_path = f"/app/data/processed/{DATASET_VERSION}/labels_back_{DATASET_VERSION}/"

COLS = 4
ROWS = 3

Path(people_save_path).mkdir(parents=True, exist_ok=True)
Path(labels_save_path).mkdir(parents=True, exist_ok=True)
Path(people_back_save_path).mkdir(parents=True, exist_ok=True)
Path(labels_back_save_path).mkdir(parents=True, exist_ok=True)

for img in tqdm.tqdm(glob.glob(people_img_path + "*")):
    
    image = cv2.imread(img)
    
    height, width, ch = image.shape
    W_STEP = width / COLS
    H_STEP = height / ROWS
    
    filename = os.path.splitext(os.path.basename(img))[0]
    
    file = open(labels_img_path + filename + ".txt", "r")
    markup = []
    for line in file.readlines():
        markup.append(yolo2bb(line, width, height))
    
    # for bbox in markup:
    #     cv2.rectangle(image,(bbox[1],bbox[2]),(bbox[3],bbox[4]),(0,255,0), 3)
    #     cv2.imwrite(f'{people_save_path}/{filename}.jpg', image)
    
    # Base crop
    crop(image, ROWS, COLS, H_STEP, W_STEP, "B", people_save_path, people_back_save_path, markup, filename)
    # Support crop 1
    crop(image, ROWS, COLS-1, H_STEP, W_STEP, "C", people_save_path, people_back_save_path, markup, filename)
    # Support crop 2
    crop(image, ROWS-1, COLS, H_STEP, W_STEP, "R", people_save_path, people_back_save_path, markup, filename)
    

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 58/58 [00:56<00:00,  1.04it/s]


### Разделение данных на обучающую и тестовую выборки

In [10]:
DATASET_VERSION = "v8"
TEST_SIZE = 0.2

dataset_path = f"/app/data/processed/{DATASET_VERSION}/"

In [11]:
files = os.listdir(dataset_path + f'images_{DATASET_VERSION}')
print(f'Общее количество изображений: {len(files)}')

Общее количество изображений: 171


In [12]:
train_images_names, test_images_names = train_test_split(files, test_size=TEST_SIZE, random_state=22)
print(f'Количество изображений для обучения: {len(train_images_names)}')
print(f'Количество изображений для тестирования: {len(test_images_names)}')

Количество изображений для обучения: 136
Количество изображений для тестирования: 35


In [19]:
def split(images, ds_type):
    Path(os.path.join(dataset_path, f'images/{ds_type}/')).mkdir(parents=True, exist_ok=True)
    Path(os.path.join(dataset_path, f'labels/{ds_type}/')).mkdir(parents=True, exist_ok=True)
    for image in images:
        shutil.copy(os.path.join(dataset_path, f'images_{DATASET_VERSION}', image), os.path.join(dataset_path, f'images/{ds_type}/'))
        shutil.copy(os.path.join(dataset_path, f'labels_{DATASET_VERSION}', image.split('.')[0]) + '.txt', os.path.join(dataset_path, f'labels/{ds_type}/'))
        
split(test_images_names, "val")
split(train_images_names, "train")

### Проверка количества данных для обучения и валидации

In [20]:
def check_splits(dataset_path):
    images_train = len(os.listdir(dataset_path + 'images/train/'))
    images_val = len(os.listdir(dataset_path + 'images/val/'))
    labels_train = len(os.listdir(dataset_path + 'labels/train/'))
    labels_val = len(os.listdir(dataset_path + 'labels/val/'))
    if images_train != labels_train:
        print(f'Разное количество данных для обучения: \
              количество изображений: {images_train} \
              количество файлов с разметкой: {labels_train}')
    else:
        print('Данные для обучения корректны.')
    if images_val != labels_val:
        print(f'Разное количество данных для валидации: \
              количество изображений: {images_val} \
              количество файлов с разметкой: {labels_val}')
    else:
        print('Данные для валидации корректны.')
    

check_splits(dataset_path)

Данные для обучения корректны.
Данные для валидации корректны.


### Добавление в датасет изображений без разметки

In [38]:
def add_back_images(ds_type):
    train_count = len(os.listdir(dataset_path + f'images/{ds_type}/'))
    train_back_count = int(train_count * 0.1)
    train_back_total = len(os.listdir(dataset_path + f'images_back_{DATASET_VERSION}'))
    train_indexes = random.sample(range(1, train_back_total), train_back_count)

    for i, train_back_image in enumerate(glob.glob(dataset_path + f'images_back_{DATASET_VERSION}/*')):
        if i in train_indexes:
            shutil.copy(train_back_image, os.path.join(dataset_path, f'images/{ds_type}/'))
            shutil.copy(".".join(train_back_image.split('.')[:-1]).replace("images","labels") + '.txt', os.path.join(dataset_path, f'labels/{ds_type}/'))
            
add_back_images("train")
add_back_images("val")

check_splits(dataset_path)

### Перемещение данных в папку для обчения

In [21]:
! mkdir /app/data/train/v8/
! mv /app/data/processed/v8/images /app/data/train/v8/
! mv /app/data/processed/v8/labels /app/data/train/v8/
! chmod -R 777 /app/data/train/v8/
! chmod -R 777 /app/data/processed/v8/

mkdir: cannot create directory ‘/app/data/train/v8/’: File exists


## Обучение
Обучение выполняется через bash скрипт `train.sh`

In [22]:
! bash /app/scripts/train.sh

Directory yolov5 is not found. Downloading...
Cloning into 'yolov5'...
remote: Enumerating objects: 14765, done.
remote: Counting objects: 100% (96/96), done.
remote: Compressing objects: 100% (58/58), done.
remote: Total 14765 (delta 53), reused 72 (delta 38), pack-reused 14669
Receiving objects: 100% (14765/14765), 13.60 MiB | 6.51 MiB/s, done.
Resolving deltas: 100% (10190/10190), done.
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice: (30 second timeout) 
wandb: W&B disabled due to login timeout.
train: weights=yolov5x.pt, cfg=, data=/app/data/train/v8/dataset.yaml, hyp=/app/yolov5/data/hyps/hyp.scratch-low.yaml, epochs=100, batch_size=2, imgsz=320, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=/app/train_logs,